# Introduction to Python

# Homework #6
# Due Monday May 9, 6pm


# Academic Honesty
* The computer science department has strict polices. Check
the department [web page](http://www.cs.columbia.edu/education/honesty) for details. 
- Do not look at anybody else's source code. Do not show anybody
your source, or leave your source where somebody could see it.
You MUST write your own code.
- For this class, feel free to discuss issues with other people, but suggest waiting an hour or two after a discussion, before writing your code.
-  Cases of non original source will be refered to the Judical Committee.



# Reminder 
- if you have already done four homeworks and are happy with them, you do NOT need to do this one

# Problem 1 - stars

# get stars data
- download and unpack [stars.zip](https://courseworks.columbia.edu/access/content/group/COMSW3101_002_2015_3/week6/stars.zip)
somewhere in your file system
- set 'basedir' to location of 'YOURPATH/stars/near'

In [1]:
basedir = '/Users/Yufei-Mac/Desktop/playandtest/HW6/stars/near'

# write loadFileProperties
- each line in file is in 'key=value' format
- value is either a float or a quoted string
- convert 'float strings' into floats

In [2]:
import os

sunpath = os.path.join(basedir, 'Sun')

with open(sunpath) as f:
    for line in f:
        print(line.strip())

ApparentMagnitude=-26.72
EffectiveTemperature=5.78e3
Constellation="Virgo"
Gravity=898.79
SpectralClass="G2V"
EscapeVelocity=1.3814e6
AlphanumericName="Sun"


In [45]:
# convert value float strings to floats 
# string values should only have one set of quotes - 
# should have 
# 'SpectralClass': 'G2V'
# not 
# 'SpectralClass': '"G2V"'
sunpath = os.path.join(basedir, 'Sun')
def loadFileProperties(path):
    dict = {}
    with open(path) as f:
        for line in f:
            a,b = line.strip().split('=')
            b = b.strip("\"")
            dict[a] = b
        return dict
            
x = loadFileProperties(sunpath)
print(x)

{'ApparentMagnitude': '-26.72', 'SpectralClass': 'G2V', 'EffectiveTemperature': '5.78e3', 'EscapeVelocity': '1.3814e6', 'Constellation': 'Virgo', 'Gravity': '898.79', 'AlphanumericName': 'Sun'}


# write stars class
- ```__init__()``` method
    - makes a sqlite3 database(in memory)
    - defines a star table in SQL
    - loads all the stars in stars/near, using loadFileProperties, into the DB
- query() method just hits SQL DB
- [sqlite3 doc](https://docs.python.org/3.5/library/sqlite3.html)

In [59]:
import os
from os import listdir
import sqlite3

class stars:

    def __init__(self, basedir):
        # makes a database that lives in memory
        self.con = sqlite3.connect(':memory:')
        
        fields = ['AlphanumericName', 'ApparentMagnitude', 'Constellation',
            'EffectiveTemperature', 'EscapeVelocity',
            'Gravity', 'SpectralClass']
        sqlTypes = ['text', 'real', 'text', 'real', 'real', 'real', 'text']

        # make star table in DB
        cur = self.con.cursor()
        query = 'create table star ('
        for a,b in zip(fields,sqlTypes):
            query = query + a +' '+b+','
        query = query[:-1]
        query += ')'
        print(query)
        cur.execute(query)
        
        # your code
        onlyfiles = [f for f in listdir(basedir)]
        querydata = []
        for x in onlyfiles:
            starpath = os.path.join(basedir, x)
            dictx = loadFileProperties(starpath)
            value = [dictx[y] for y in fields]
            querydata.append(value)
        cur.executemany('insert into star values(?, ?, ?, ?, ?, ?, ?)', querydata)
    def query(self, q):
        return self.con.execute(q)

In [60]:
# print how many stars you load - should get 55

s=stars(basedir)

create table star (AlphanumericName text,ApparentMagnitude real,Constellation text,EffectiveTemperature real,EscapeVelocity real,Gravity real,SpectralClass text)


In [61]:
list(s.query('select AlphanumericName,Gravity,EscapeVelocity from star where Gravity>2400'))

[('GJ 1061', 2800.0, 1200000.0),
 ('GJ 1245 A', 2800.0, 1200000.0),
 ('Gl 15 B', 3000.0, 1200000.0),
 ('Gl 473 A', 2800.0, 1200000.0),
 ('Gl 752 B', 2600.0, 1200000.0),
 ('Gl 860 B', 3000.0, 1200000.0),
 ('Gl 866 A', 2600.0, 1200000.0),
 ('Gl 905', 3000.0, 1200000.0),
 ('HIP 5643', 2800.0, 1200000.0),
 ('Luyten 726-8 A', 2800.0, 1200000.0),
 ('Luyten 726-8 B', 2800.0, 1200000.0),
 ("Luyten's Star", 2600.0, 1200000.0),
 ('Proxima Centauri', 2600.0, 1200000.0),
 ('Sirius B', 12000000.0, 15000000.0),
 ('Wolf 359', 3000.0, 1200000.0)]

In [63]:
list(s.query('select avg(Gravity), avg(EscapeVelocity) from star where Gravity>2400'))

[(802613.3333333334, 2120000.0)]

# Problem 2 - interleave
- takes two strings and computes every possible interleave
- an interleave uses all the characters from each string once, and the order of each string is preserved in the interleaved string
- each input string is in alphabetical order, with no duplicated characters
- first arg is all lower case, second arg is all upper case
- many ways to do this
- one straightforward approach is to concat the two strings, and apply itertools.permutations(). this will generate a list containing all the interleaved strings, plus bogus ones. just filter out the bogus strings, and return what's left
- in a bogus string the lower case and/or upper case letters are not in the original order

In [80]:
import itertools
def interleave(lower,upper):
    result = []
    CombineString = lower+upper
    AllResult = itertools.permutations(CombineString)
    for x in AllResult:
        testupper = ''.join(y for y in x if y.isupper())
        testlower = ''.join(y for y in x if y.islower())
        if testlower == lower and testupper == upper:
            result.append(''.join(x))
    #print (result)
    return result
interleave('abcde', 'X')

['abcdeX', 'abcdXe', 'abcXde', 'abXcde', 'aXbcde', 'Xabcde']

In [81]:
# 'baXY', 'YXab', 'baYX' are bogus, because the order of 
# one or both of the lower and upper case strings has not
# been maintained

interleave('ab', 'XY')

['abXY', 'aXbY', 'aXYb', 'XabY', 'XaYb', 'XYab']

In [82]:
interleave('abc', 'XYZ')

['abcXYZ',
 'abXcYZ',
 'abXYcZ',
 'abXYZc',
 'aXbcYZ',
 'aXbYcZ',
 'aXbYZc',
 'aXYbcZ',
 'aXYbZc',
 'aXYZbc',
 'XabcYZ',
 'XabYcZ',
 'XabYZc',
 'XaYbcZ',
 'XaYbZc',
 'XaYZbc',
 'XYabcZ',
 'XYabZc',
 'XYaZbc',
 'XYZabc']

# Problem 3 - zipfiles
- Extract data directly from a zipfile without unpacking it
- use zipfile module [doc](https://docs.python.org/3.5/library/zipfile.html#zipfile.ZipInfo). only need three things
    - zipfile.ZipFile()
    - zipfile.namelist()
    - zipfile.open()
- extract info from python's help documentation
    - on mac, clicking on the zip link will automatically unpack, which we don't want. instead right click on the link, and do 'Download Linked File as...' instead
- zip file to download [zip of python docs in plain text format](https://docs.python.org/3/archives/python-3.5.1-docs-text.zip)

In [283]:
# set to where ever you place the python docs zip

zfile = '/Users/lstead/Downloads/python-3.5.1-docs-text.zip'

# write help class
- ```__init__()``` method should read in all the help files and save
them in the help object. remember to convert to unicode, and remove new lines
    - hits() method - checks all file lines for match, and return hit count
    - grep() method - print file name, line number, and line of each match
    - files() method - return dict of file/# of hits

In [258]:
h = help(zfile)

reading 477 files


In [131]:
h.files('zipfiles')

defaultdict(int,
            {'python-3.5.1-docs-text/distutils/apiref.txt': 1,
             'python-3.5.1-docs-text/library/warnings.txt': 1,
             'python-3.5.1-docs-text/reference/import.txt': 1,
             'python-3.5.1-docs-text/using/cmdline.txt': 3,
             'python-3.5.1-docs-text/whatsnew/3.2.txt': 1,
             'python-3.5.1-docs-text/whatsnew/changelog.txt': 1})

In [118]:
[h.hits(w) for w in ['rpartition', 'linux','universal', 'mode', 'zipfile', 'zipfiles']]

[10, 41, 67, 1103, 78, 8]

In [99]:
h.grep('rpartition')

python-3.5.1-docs-text/whatsnew/2.7.txt:917: * The "split()", "replace()", "rindex()", "rpartition()", and
python-3.5.1-docs-text/library/stdtypes.txt:1519: str.rpartition(sep)
python-3.5.1-docs-text/library/stdtypes.txt:2275: bytes.rpartition(sep)
python-3.5.1-docs-text/library/stdtypes.txt:2276: bytearray.rpartition(sep)
python-3.5.1-docs-text/whatsnew/2.5.txt:976: "rpartition(sep)" methods that simplify a common use case.
python-3.5.1-docs-text/whatsnew/2.5.txt:985: "rpartition(sep)" also returns a 3-tuple but starts searching from
python-3.5.1-docs-text/whatsnew/2.5.txt:996: >>> 'www.python.org'.rpartition('.')
python-3.5.1-docs-text/whatsnew/2.5.txt:998: >>> 'www.python.org'.rpartition(':')
python-3.5.1-docs-text/whatsnew/3.2.txt:2368: extension = name.rpartition('.')[2]
python-3.5.1-docs-text/whatsnew/3.2.txt:2409: also used by "rfind()", "rindex()", "rsplit()" and "rpartition()".


In [103]:
h.grep('zipfiles')

python-3.5.1-docs-text/library/warnings.txt:322: used to support displaying source for modules found in zipfiles or
python-3.5.1-docs-text/distutils/apiref.txt:794: as tarballs or zipfiles.
python-3.5.1-docs-text/using/cmdline.txt:83: and zipfiles that are passed to the interpreter as the script
python-3.5.1-docs-text/using/cmdline.txt:430: may refer to zipfiles containing pure Python modules (in either
python-3.5.1-docs-text/using/cmdline.txt:432: zipfiles.
python-3.5.1-docs-text/whatsnew/changelog.txt:2278: * Issue #20078: Reading malformed zipfiles no longer hangs with 100%
python-3.5.1-docs-text/whatsnew/3.2.txt:1531: zipfiles, uncompressed tarfiles, gzipped tarfiles, and bzipped
python-3.5.1-docs-text/reference/import.txt:653: libraries) from zipfiles.


In [168]:
h.files('zipfiles')

defaultdict(int,
            {'python-3.5.1-docs-text/distutils/apiref.txt': 1,
             'python-3.5.1-docs-text/library/warnings.txt': 1,
             'python-3.5.1-docs-text/reference/import.txt': 1,
             'python-3.5.1-docs-text/using/cmdline.txt': 3,
             'python-3.5.1-docs-text/whatsnew/3.2.txt': 1,
             'python-3.5.1-docs-text/whatsnew/changelog.txt': 1})

# Problem 4 - sumTrianglePath

# Write str2lists 
- converts string form of triangle data into nested lists
- homework-6.py has the data in ascii

In [162]:
def str2lists(triangle):
    level = 1
    current = 0
    result = []
    oneline = []
    for x in triangle.split():
        if current>=level:
            current = 0
            level += 1
            result.append(oneline)
            oneline = []
        oneline.append(int(x))
        current += 1
        
    result.append(oneline)
    return result

s1='''
   3
  7 4
 2 4 6
'''
x1= str2lists(s1)
x1

[[3], [7, 4], [2, 4, 6]]

In [163]:
s2='''
   3
  7 4
 2 4 6
8 5 9 3
'''
x2 = str2lists(s2)
x2

[[3], [7, 4], [2, 4, 6], [8, 5, 9, 3]]

In [164]:
s3 = '''
                   75
                 95 64
                17 47 82
              18 35 87 10
             20 04 82 47 65
            19 01 23 75 03 34
          88 02 77 73 07 63 67
         99 65 04 28 06 16 70 92
        41 41 26 56 83 40 80 70 33
       41 48 72 33 47 32 37 16 94 29
      53 71 44 65 25 43 91 52 97 51 14
     70 11 33 28 77 73 17 78 39 68 17 57
    91 71 52 38 17 14 91 43 58 50 27 29 48
  63 66 04 68 89 53 67 30 73 16 69 87 40 31
04 62 98 27 23 09 70 98 73 93 38 53 60 04 23
'''

x3 = str2lists(s3)
x3

[[75],
 [95, 64],
 [17, 47, 82],
 [18, 35, 87, 10],
 [20, 4, 82, 47, 65],
 [19, 1, 23, 75, 3, 34],
 [88, 2, 77, 73, 7, 63, 67],
 [99, 65, 4, 28, 6, 16, 70, 92],
 [41, 41, 26, 56, 83, 40, 80, 70, 33],
 [41, 48, 72, 33, 47, 32, 37, 16, 94, 29],
 [53, 71, 44, 65, 25, 43, 91, 52, 97, 51, 14],
 [70, 11, 33, 28, 77, 73, 17, 78, 39, 68, 17, 57],
 [91, 71, 52, 38, 17, 14, 91, 43, 58, 50, 27, 29, 48],
 [63, 66, 4, 68, 89, 53, 67, 30, 73, 16, 69, 87, 40, 31],
 [4, 62, 98, 27, 23, 9, 70, 98, 73, 93, 38, 53, 60, 4, 23]]

# Write sumTrianglePath 
- takes nested list triangle represention and returns the max path
- starting at the top, and moving down to adjacent numbers on the row below, find
the path with the maximum sum
- hint: this is very simple
    - keep merging the last row of the triangle into the 2nd to last row, until you are done
    - you may find it convenient to make a copy of the input, use [copy.deepcopy](https://docs.python.org/3.5/library/copy.html)
    - don't need recursion
- in triangle below, the max sum from top to bottom is 3+7+4+9=23

```
   3
  7 4
 2 4 6
8 5 9 3
```

In [167]:
import copy
def sumTrianglePath(x):
    xcp = copy.deepcopy(x)
    for i in reversed(range(1,len(x))):
        for j in range(len(xcp[i])-1):
            xcp[i-1][j] = xcp[i-1][j] + max(xcp[i][j],xcp[i][j+1])
    return xcp[0][0]

[sumTrianglePath(x) for x in [x1, x2, x3]]

[14, 23, 1074]

# Problem 5 - Conway's game of life
- the ```__init__()``` method takes as input an n x n numpy array of 1s and 0s, 
    representing different cell states. A cell
    is considered alive if it is of state 1, and dead otherwise. The neighbors of
    a cell are any of the cells that are adjacent to it, 3 at a corner, 5 on 
    an edge, 8 in the interior. 
- the ```__next__()``` method operates on the current board, and returns
    the next state of the board as determined by the following rules:
    - If a cell is alive, and it has exactly two or three neighbors that is also alive,
    then in the next turn, it will remain alive.
    - If a cell is alive and it has less than two or more than three alive neighbors, it will die.
    - If a cell is dead and it has exactly three neighbors which are alive, then next turn it will
    come back to life. Otherwise, it remains dead.
- use 2d numpy array internally
- you must complete the ```__next__()``` and countAliveNeighbors() methods
- many interesting patterns have been discovered
- [wiki article](https://en.wikipedia.org/wiki/Conway%27s_Game_of_Life)

In [201]:
import numpy as np

class life:
    def __init__(self, board):
        # current board - will be updated by __next__
        self.current = np.array(board)
        self.n = len(board)
    
    def run(self, n):
        self.plife()
        for j in range(n):
            next(self)
            self.plife()

    def plife(self):
        s = []
        for row in range(self.n):
            for col in range(self.n):
                s.append('1' if self.current[row,col] else '0')
            s.append('\n')
        print(''.join(s))
        
    def __next__(self):
        outputBoard = self.current.copy()

        # compute and store new board in outputBoard
        
        # Your code here
        for i in range(self.n):
            for j in range(self.n):
                count = self.countAliveNeighbors(i, j)
                if count==3:
                    outputBoard[i][j] = 1
                elif count==2:
                    pass
                else:
                    outputBoard[i][j] = 0
        self.current = outputBoard
        return outputBoard

    def countAliveNeighbors(self, i, j):
        
        # don't go off the board on edges
        count = 0
        for x in range(max(i-1,0),min(i+2,self.n)):
            for y in range(max(j-1,0),min(j+2,self.n)):
                count += self.current[x][y]
        # Your code here
        count -= self.current[i][j]
        return count


In [202]:
# homework-6.py has the data in ascii

# stable, doesn't change

block = life([[0,0,0,0],
             [0,1,1,0],
             [0,1,1,0],
             [0,0,0,0]])

In [203]:
block.run(2)

0000
0110
0110
0000

0000
0110
0110
0000

0000
0110
0110
0000



In [204]:
# aternates between two patterns

blinker = life([[0, 1, 0 ],
          [0, 1, 0],
          [0, 1, 0]])

blinker.run(4)

010
010
010

000
111
000

010
010
010

000
111
000

010
010
010



In [205]:
toad = life(np.array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 1, 1, 1, 0],
       [0, 1, 1, 1, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]]))

toad.run(2)

000000
000000
001110
011100
000000
000000

000000
000100
010010
010010
001000
000000

000000
000000
001110
011100
000000
000000



In [206]:
everybodydies = life(np.array([
            [1,1,1,0,1],
            [1,0,0,1,0],
            [0,1,0,0,1],
            [1,1,0,1,0],
            [1,0,1,0,1],
            ]))
everybodydies.run(16)

11101
10010
01001
11010
10101

11110
10011
01011
10011
10110

11111
10000
11000
10000
01111

11110
00010
11000
10010
01110

01110
00010
11100
10010
01110

00110
10010
11110
10010
01110

00110
10001
10011
10001
01110

00010
01101
11011
10001
01110

00110
11001
10001
10001
01110

01110
11101
10011
10101
01110

10010
10001
10001
10001
01110

00000
11011
11011
10101
01110

00000
11011
00000
10001
01110

00000
00000
11011
01110
01110

00000
00000
11011
00000
01010

00000
00000
00000
11011
00000

00000
00000
00000
00000
00000



In [207]:
fig8 = life(np.array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]))

fig8.run(8)

000000000000
000000000000
000000000000
000111000000
000111000000
000111000000
000000111000
000000111000
000000111000
000000000000
000000000000
000000000000

000000000000
000000000000
000010000000
000101000000
001000100000
000100010000
000010001000
000001000100
000000101000
000000010000
000000000000
000000000000

000000000000
000000000000
000010000000
000111000000
001110100000
000100010000
000010001000
000001011100
000000111000
000000010000
000000000000
000000000000

000000000000
000000000000
000111000000
001000000000
001000100000
001001010000
000010100100
000001000100
000000000100
000000111000
000000000000
000000000000

000000000000
000010000000
000110000000
001011000000
011100100000
000101010000
000010101000
000001001110
000000110100
000000011000
000000010000
000000000000

000000000000
000110000000
000000000000
010001000000
010000100000
000101010000
000010101000
000001000010
000000100010
000000000000
000000011000
000000000000

000000000000
000000000000
000010000000
000000000000
001011